In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('C:\\Sahil\\NLP\\hindi_reconstructed_sentences2.csv')

In [3]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True) 

In [4]:
df_val = df.iloc[:1000]
df_test= df.iloc[1000:2000]
df_train = df.iloc[2000:1000000]

In [5]:
from collections import defaultdict

In [6]:
unigrams = defaultdict(int)
bigrams = defaultdict(int)
trigrams = defaultdict(int)
quadrigrams = defaultdict(int)

In [8]:
N=0
for sentence in df_train['reconstructed_sentence']:
    tokens = ['<s>','<s>','<s>'] + sentence.split() + ['</s>']
    N += len(tokens) - 3  
    for i in range(3, len(tokens)):
        w1, w2, w3, w4 = tokens[i-3], tokens[i-2], tokens[i-1], tokens[i]
        unigrams[w4] += 1
        bigrams[(w3, w4)] += 1
        trigrams[(w2, w3, w4)] += 1
        quadrigrams[(w1, w2, w3, w4)] += 1

In [ ]:
lambda_counts = [0, 0, 0, 0]  

for sentence in df_val['reconstructed_sentence']:
    tokens = ['<s>','<s>','<s>'] + sentence.split() + ['</s>']
    for i in range(3, len(tokens)):
        w1, w2, w3, w4 = tokens[i-3], tokens[i-2], tokens[i-1], tokens[i]

      
        p_quad = (quadrigrams.get((w1,w2,w3,w4),0) - 1) / max(quadrigrams.get((w1,w2,w3),1) - 1, 1)
        p_tri  = (trigrams.get((w2,w3,w4),0) - 1) / max(trigrams.get((w2,w3),1) - 1, 1)
        p_bi   = (bigrams.get((w3,w4),0) - 1) / max(bigrams.get((w3,),1) - 1, 1)
        p_uni  = (unigrams.get(w4,0) - 1) / max(N-1,1)

        probs = [p_uni, p_bi, p_tri, p_quad]
        max_index = probs.index(max(probs))
        lambda_counts[max_index] += 1

In [10]:
lambdas = [lc / sum(lambda_counts) for lc in lambda_counts]
print("Best λ parameters (uni, bi, tri, quad):", lambdas)

Best λ parameters (uni, bi, tri, quad): [0.15675731568607176, 0.8432426843139282, 0.0, 0.0]


In [11]:
def interpolated_prob(w1, w2, w3, w4):
    p_quad = quadrigrams.get((w1,w2,w3,w4),0) / max(quadrigrams.get((w1,w2,w3),1),1)
    p_tri  = trigrams.get((w2,w3,w4),0) / max(trigrams.get((w2,w3),1),1)
    p_bi   = bigrams.get((w3,w4),0) / max(bigrams.get((w3,),1),1)
    p_uni  = unigrams.get(w4,0) / max(N,1)

    return lambdas[0]*p_uni + lambdas[1]*p_bi + lambdas[2]*p_tri + lambdas[3]*p_quad


In [ ]:
for sentence in df_test['reconstructed_sentence']:
    tokens = ['<s>','<s>','<s>'] + sentence.split() + ['</s>']
    sentence_prob = 1.0

    for i in range(3, len(tokens)):
        w1, w2, w3, w4 = tokens[i-3], tokens[i-2], tokens[i-1], tokens[i]
        p = interpolated_prob(w1, w2, w3, w4)
        sentence_prob *= p 

    print(f"Interpolated probability for '{sentence}': {sentence_prob:.6f}")


Interpolated probability for 'करीब  दो  माह  पूर्व  विक्की  राजगढ़  आया  था ।': 16303253.206653
Interpolated probability for 'आपको  सही  जवाब  का  ऑप्शन  टाइप  कर  ये  एसएमएस  509093 पर  भेजना  होगा ।': 0.000000
Interpolated probability for 'वहीं  रात  में  ही  मुठभेड़  के  दौरान  उसका  साथी  अंधेरे  में  भाग  निकला ।': 441066708642453150574407530710040576.000000
Interpolated probability for 'येलो  टी  बनाने  के  लिए , आपको  थोड़ा  पानी  गर्म  करने  की  जरुरत  है ।': 18861737788957796120680581858066432.000000
Interpolated probability for 'सुबह  10 बजे  से  11 बजे  तक  नामाकंन , 11 बजे  से  12 बजे  तक  नामांकन  पत्रों  की  जांच , 12 बजे  से  साढे  12 बजे  तक  नाम  वापसी , साढे  12 बजे  से  अपराह्न  1 बजे  तक  चुनाव  चिन्ह  आवंटन , एक  बजे  से  तीन  बजे  तक  मतदान  व  इसके  बाद  मतगणना  की  जाएगी ।': 15439153821231353139472029398296679142382347472636626551951658383606006473276188694632747244206983349272576.000000
Interpolated probability for 'जिसमें  उन्होंने  कहा , बीजेपी  की  अगुवाई  व

In [2]:
import pandas as pd

In [3]:
df_uni_prob=pd.read_csv("C:\\Sahil\\NLP\\Lab4\\unigram1000senProb.csv")
df_bi_prob=pd.read_csv("C:\\Sahil\\NLP\\Lab4\\bigram1000senProb.csv")
df_tri_prob=pd.read_csv("C:\\Sahil\\NLP\\Lab4\\trigram1000senProb.csv")
df_quad_prob=pd.read_csv("C:\\Sahil\\NLP\\Lab4\\quadgram1000senProb.csv")

In [4]:
df_uni_prob.head()

,Sentence,Add-One,Add-K,Token-Type
0,उसे तत्काल ट्रेन के शौचालय में बंद कर ...,9.400667e-71,1.007157e-70,2.660026e-136
1,ऐसे ही एक कार्यक्रम का एक वीडियो सोशल ...,6.091513e-38,6.362870e-38,2.168721e-82
2,आरबीजीडी की टीम ने टास जीत कर पहले खेल...,6.903272e-70,6.105506e-70,7.225952e-119
3,इससे आम आदमी की गाढ़ी कमाई को फर्जी और ...,1.216693e-61,1.269652e-61,9.723639e-112
4,विधि शिक्षा एवं विधि सहायता केन्द्र सेंट...,1.391549e-131,1.556501e-131,1.159317e-220


In [5]:
df_bi_prob.head()

,Sentence,Add-One,Add-K,Token-Type
0,उसे तत्काल ट्रेन के शौचालय में बंद कर ...,5.108519e-73,3.743680e-68,4.756580e-146
1,ऐसे ही एक कार्यक्रम का एक वीडियो सोशल ...,5.896634e-30,4.154974e-27,8.973592e-91
2,आरबीजीडी की टीम ने टास जीत कर पहले खेल...,2.469219e-71,5.484179e-67,6.861254e-128
3,इससे आम आदमी की गाढ़ी कमाई को फर्जी और ...,1.530544e-61,7.968936e-58,3.853505e-121
4,विधि शिक्षा एवं विधि सहायता केन्द्र सेंट...,4.295161e-137,8.160860e-129,4.408617e-231


In [6]:
import numpy as np

In [7]:

df_test_prob=pd.read_csv('test_with_probs.csv')

In [8]:
df_val_prob=pd.read_csv('val_with_probs.csv')

In [9]:
for i,key in df_test_prob.iterrows():
    print(key)
    

reconstructed_sentence    करीब  दो  माह  पूर्व  विक्की  राजगढ़  आया  था ।
Prob_bigram                                                   -132.043031
Prob_trigram                                                  -148.756527
Prob_quadrigram                                               -153.908786
Prob_unigram                                                   -70.597463
Name: 0, dtype: object
reconstructed_sentence    आपको  सही  जवाब  का  ऑप्शन  टाइप  कर  ये  एसएम...
Prob_bigram                                                     -228.412662
Prob_trigram                                                    -309.068079
Prob_quadrigram                                                 -303.941232
Prob_unigram                                                    -118.532916
Name: 1, dtype: object
reconstructed_sentence    वहीं  रात  में  ही  मुठभेड़  के  दौरान  उसका  ...
Prob_bigram                                                     -159.162901
Prob_trigram                                        

In [10]:
prob_for_lam_val={}

In [11]:
for lam1 in np.arange(0,1,0.01):
    for lam2 in np.arange(0,1,0.01): 
        for lam3 in np.arange(0,1,0.01):
            for lam4 in np.arange(0,1,0.01):
                probsum=0
                if (lam1+lam2+lam3+lam4)==1:
                    for i,key in df_test_prob.iterrows():
                        probsum+=(-1*((lam1*key['Prob_unigram'])+(lam2*key['Prob_bigram'])+(lam3*key['Prob_trigram'])+(lam4*key['Prob_quadrigram'])))
                prob_for_lam_val[lam1,lam2,lam3,lam4]=probsum


In [ ]:

max_item = max(prob_for_lam_val.items(), key=lambda kv: kv[1])
min_item = min(prob_for_lam_val.items(), key=lambda kv: kv[1])

max_lams, max_val = max_item
min_lams, min_val = min_item

print("MAX probsum = {:.6f} at lam1..lam4 = {}".format(max_val, max_lams))
print("MIN probsum = {:.6f} at lam1..lam4 = {}".format(min_val, min_lams))


MAX probsum = 386893.519267 at lam1..lam4 = (np.float64(0.0), np.float64(0.0), np.float64(0.01), np.float64(0.99))
MIN probsum = 0.000000 at lam1..lam4 = (np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0))
